In [ ]:
import numpy as np
import matplotlib.pyplot as plot

VIS_DIR = "runs/"
BENCH_DIR = "exports/"

The following cell loads the data to be used in the following 3 cells.

In [ ]:
# Read graph and one query (optional) from file
GRAPH_FILE = "graph.txt"
QUERIES_FILE = "queries.txt"
PLOT_EDGES = True
PLOT_QUERY_INDEX = 3

mode = -1
# Current layer being read for a node
reading_layer = -1
node_layers = {}
nodes_by_layer = []
connections_by_layer = []
with open(VIS_DIR + GRAPH_FILE, "r") as f:
    # Initialize dicts for each layer
    num_layers = int(f.readline().strip())
    for i in range(num_layers):
        nodes_by_layer.append({})
        connections_by_layer.append({})

    for line in f.readlines():
        line = line.strip()
        if line == "Nodes":
            mode = 0
            continue
        elif line == "Edges":
            mode = 1
            continue
        
        if mode == 0:
            # Read node
            node_dims = [float(x) for x in line.split(":")[1].split(",")]
            node_id = int(line.split(":")[0].split(" ")[0])
            layer = int(line.split(":")[0].split(" ")[1])
            for i in range(layer + 1):
                nodes_by_layer[i][node_id] = node_dims
            node_layers[node_id] = layer
        elif mode == 1:
            if reading_layer == -1:
                # Read node index
                node_id = int(line)
                layer = node_layers[node_id]
                reading_layer = 0
            else:
                # Read edges for layer
                if line != "":
                    if line[-1] == ",":
                        line = line[:-1]
                    connections_by_layer[reading_layer][node_id] = [int(x) for x in line.split(",")]
                if reading_layer == layer:
                    reading_layer = -1
                else:
                    reading_layer += 1

query = None
query_neighbors = None
query_path = None
if PLOT_QUERY_INDEX >= 0:
    reading_query = -1
    with open(VIS_DIR + QUERIES_FILE, "r") as f:
        for line in f.readlines():
            line = line.strip()
            
            if line == "Query {}".format(PLOT_QUERY_INDEX):
                reading_query = 0
                continue
            if line[-1] == ",":
                line = line[:-1]

            if reading_query == 0:
                query = [float(x) for x in line.split(",")]
                reading_query = 1
            elif reading_query == 1:
                query_neighbors = [int(x) for x in line.split(",")]
                reading_query = 2
            elif reading_query == 2:
                query_path = [int(x) for x in line.split(",")]
                reading_query = -1
                break

The following cell allows for graphing nodes by layer, as well as the closest node for a single query on every layer.

In [ ]:
# Plot nodes as 2D points by layer
EXPORT_PNG = False

for i in range(len(nodes_by_layer)):
    layer_nodes = nodes_by_layer[i]
    node_dims = np.array(list(layer_nodes.values()))
    plot.title("Layer " + str(i))
    plot.scatter(node_dims[:, 0], node_dims[:, 1])

    if i == 0:
        min_x, max_x = plot.gca().get_xlim()
        min_y, max_y = plot.gca().get_ylim()
    else:
        plot.xlim(min_x, max_x)
        plot.ylim(min_y, max_y)

    # Plot edges
    connections = connections_by_layer[i]
    if PLOT_EDGES:
        for node_id in connections:
            for connection in connections[node_id]:
                width, height = max_x - min_x, max_y - min_y
                plot.arrow(layer_nodes[node_id][0], layer_nodes[node_id][1], layer_nodes[connection][0] - layer_nodes[node_id][0], layer_nodes[connection][1] - layer_nodes[node_id][1],
                        color='green', alpha=0.3, head_width=width / 35, head_length=height / 35, length_includes_head=True)
            
    if PLOT_QUERY_INDEX >= 0:
        plot.scatter(query[0], query[1], color='red', marker='x')
        if i > 0:
            # Plot entry point
            point = query_path[len(nodes_by_layer) - 1 - i]
            plot.scatter(layer_nodes[point][0], layer_nodes[point][1], color='red', marker='o', facecolors='none')
        else:
            # Plot query neighbors
            for neighbor in query_neighbors:
                plot.scatter(layer_nodes[neighbor][0], layer_nodes[neighbor][1], color='yellow', marker='x')

    if EXPORT_PNG:
        plot.savefig(VIS_DIR + "Layer " + str(i) + ".png")
    plot.show()

The following two cells allow for visualizing HNSW search on the bottommost layer.

In [ ]:
# Read a specific query from file
QUERY_SEARCH_FILE = "query_search.txt"

query_search = {}
reading_query = -1
iteration = -1
with open(VIS_DIR + QUERY_SEARCH_FILE, "r") as f:
    for line in f.readlines():
        line = line.strip()
        if line[-1] == ",":
            line = line[:-1]

        if line.startswith("Iteration "):
            iteration = int(line.split(" ")[1])
            query_search[iteration] = {}
            reading_query = 0
        elif reading_query == 0:
            query_search[iteration]["visit"] = [int(x) for x in line.split(",")]
            reading_query = 1
        elif reading_query == 1:
            query_search[iteration]["cand"] = [int(x) for x in line.split(",")]
            reading_query = 2
        elif reading_query == 2:
            query_search[iteration]["found"] = [int(x) for x in line.split(",")]
            reading_query = -1

In [ ]:
# Iterate over query_search pairs
PLOT_EDGES = True
EXPORT_PNG = False
ITERATION_SKIPS = 1

for itr in query_search:
    if itr % ITERATION_SKIPS != 0 and itr != iteration:
        continue

    layer_nodes = nodes_by_layer[0]
    node_dims = np.array(list(layer_nodes.values()))
    plot.title("Iteration " + str(itr))
    plot.scatter(node_dims[:, 0], node_dims[:, 1])

    min_x, max_x = plot.gca().get_xlim()
    min_y, max_y = plot.gca().get_ylim()

    # Plot edges
    connections = connections_by_layer[0]
    if PLOT_EDGES:
        for node_id in connections:
            for connection in connections[node_id]:
                width, height = max_x - min_x, max_y - min_y
                plot.arrow(layer_nodes[node_id][0], layer_nodes[node_id][1], layer_nodes[connection][0] - layer_nodes[node_id][0], layer_nodes[connection][1] - layer_nodes[node_id][1],
                        color='green', alpha=0.3, head_width=width / 35, head_length=height / 35, length_includes_head=True)
            
    # Plot query search
    for node_id in query_search[itr]["visit"]:
        plot.scatter(layer_nodes[node_id][0], layer_nodes[node_id][1], color='red', marker='o', facecolors='none')
    for node_id in query_search[itr]["cand"]:
        width, height = max_x - min_x, max_y - min_y
        plot.scatter(layer_nodes[node_id][0], layer_nodes[node_id][1], color='blue', marker='s', s=100, facecolors='none')
    for node_id in query_search[itr]["found"]:
        plot.scatter(layer_nodes[node_id][0], layer_nodes[node_id][1], color='yellow', marker='x')
    
    if EXPORT_PNG:
        plot.savefig(VIS_DIR + "Iteration " + str(itr) + ".png")
    plot.show()

The following cell allows for visualizating recall and distance computations for a HNSW benchmark run.

In [ ]:
QUERIES_LIST = [100]
RESULTS_LIST = [20]
GRAPH_NAME = "random_graph"
GRAPH_ID_LIST = [0, 1, 2]

EXPORT_PNG = False

for id in GRAPH_ID_LIST:
    for i in range(len(QUERIES_LIST)):
        csv_name = GRAPH_NAME + "_results_{}_{}_{}.csv".format(QUERIES_LIST[i], RESULTS_LIST[i], id)
        with open(BENCH_DIR + csv_name, 'r') as f:
            # Read title
            title = f.readline()
            f.readline()

            # Read lines and convert each line to a list of floats
            graph_line = [line.split(',') for line in f.readlines()]
            graph_line = [[float(value) for value in point] for point in graph_line]

            # Graph line in plot
            graph_line = np.array(graph_line)
            plot.plot(graph_line[:,1], graph_line[:,2], 'o-')

    plot.legend(["{} queries, {} results".format(QUERIES_LIST[i], RESULTS_LIST[i]) for i in range(len(QUERIES_LIST))])
    plot.xlabel("Distance Computations")
    plot.ylabel("Recall")
    plot.title(title)
    if EXPORT_PNG:
        plot.savefig(BENCH_DIR + GRAPH_NAME + "_results_{}.png".format(id))
    plot.show()

The following two cells are used to visualize recalls and distance computations for a single HNSW run.

In [ ]:
RECALL_PER_QUERY_FILE = "indiv.txt"
EF_SEARCH_VALUE = 100
TITLE = "Recall distribution for EF_SEARCH = {}".format(EF_SEARCH_VALUE)
EXPORT_PNG = False

query_recalls = []

# Read recall per query file
with open(VIS_DIR + RECALL_PER_QUERY_FILE, 'r') as f:
    # Each line contains a recall value and distance comps
    for line in f.readlines():
        line = line.strip()
        query_recalls.append(float(line.split(" ")[0]))

# Plot frequency distribution
plot.hist(query_recalls, bins=100)
plot.xlabel("Recall")
plot.ylabel("Frequency")
plot.title(TITLE)
if EXPORT_PNG:
    plot.savefig(VIS_DIR + "recall_dist_{}.png".format(EF_SEARCH_VALUE))
plot.show()

In [ ]:
RECALL_PER_QUERY_FILE = "indiv.txt"
EF_SEARCH_VALUE = 100
TITLE = "Recall vs distance comps for EF_SEARCH = {}".format(EF_SEARCH_VALUE)
EXPORT_PNG = False

values = []

# Read recall per query file
with open(VIS_DIR + RECALL_PER_QUERY_FILE, 'r') as f:
    # Each line contains a recall value and distance comps
    for line in f.readlines():
        line = line.strip()
        values.append([float(x) for x in line.split(" ")])

# Plot scatterplot: recall vs distance comps
values = np.array(values)
plot.scatter(values[:,1], values[:,0])
plot.xlabel("Distance Computations")
plot.ylabel("Recall")
plot.title(TITLE)
if EXPORT_PNG:
    plot.savefig(VIS_DIR + "recall_vs_dist_{}.png".format(EF_SEARCH_VALUE))
plot.show()

The following cell is for visualizing at what distance computations the neighbors are found.

In [ ]:
WHEN_NEIGH_FOUND_FILE = "when_neigh_found.txt"
HISTOGRAM_INTERVAL = 500
HISTOGRAM_BINS = 10
# Indices ((n+1)th closest neighbor) to plot (blank for all)
indices = []
EXPORT_PNG = False

lines = []

# Read when neighbors are found file
with open(VIS_DIR + WHEN_NEIGH_FOUND_FILE, 'r') as f:
    # Read distance computations for each neighbor
    for line in f.readlines():
        line = line.strip()
        lines.append([int(x) for x in line.split(" ")])

not_found = 0
# Group distance computations into histogram bins
# Have a bin for values greater than the last bin
hist = np.zeros(HISTOGRAM_BINS + 1)
for line in lines:
    for i in range(len(line)):
        if len(indices) == 0 or i in indices:
            if line[i] == -1:
                not_found += 1
            else:
                hist[min(line[i] // HISTOGRAM_INTERVAL, HISTOGRAM_BINS)] += 1

# Plot bar histogram excluding last bin
plot.bar([x * HISTOGRAM_INTERVAL for x in range(HISTOGRAM_BINS)], hist[:-1], width=HISTOGRAM_INTERVAL)
plot.xlabel("Distance Computations")
plot.ylabel("Frequency")
plot.title("When neighbors are found")

if EXPORT_PNG:
    plot.savefig(VIS_DIR + "when_neigh_found.png")
plot.show()

# Display table of when neighbors are found
print("When neighbors are found")
print("Distance Comps\tFrequency")
for i in range(len(hist) - 1):
    print("{}\t{}".format(i * HISTOGRAM_INTERVAL, int(hist[i])))
# Print out last bin
print("{}+\t{}".format((len(hist) - 1) * HISTOGRAM_INTERVAL, int(hist[-1])))
# Print total
print("Total found\t{}".format(int(sum(hist))))
# Print not found
print("Not found\t{}".format(not_found))

# Print average and median of all values
values = []
for line in lines:
    for i in range(len(line)):
        if len(indices) == 0 or i in indices:
            if line[i] != -1:
                values.append(line[i])

print("Average\t{}".format(sum(values) / len(values)))
print("Median\t{}".format(np.median(values)))

The following cell is for visualizing the correlation between nth nearest neighbors.

In [ ]:
from sklearn.linear_model import LinearRegression

WHEN_NEIGH_FOUND_FILE = "when_neigh_found.txt"
FIRST_INDEX = 0
SECOND_INDEX = 19
EXPORT_PNG = False

lines = []

# Read when neighbors are found file
with open(VIS_DIR + WHEN_NEIGH_FOUND_FILE, 'r') as f:
    # Read distance computations for each neighbor
    for line in f.readlines():
        line = line.strip()
        lines.append([int(x) for x in line.split(" ")])

x = np.array([line[FIRST_INDEX] for line in lines if line[FIRST_INDEX] != -1 and line[SECOND_INDEX] != -1])
y = np.array([line[SECOND_INDEX] for line in lines if line[FIRST_INDEX] != -1 and line[SECOND_INDEX] != -1])

# Plot scatterplot
plot.scatter(x, y)
plot.xlabel("Distance Computations for neighbor {}".format(FIRST_INDEX + 1))
plot.ylabel("Distance Computations for neighbor {}".format(SECOND_INDEX + 1))
plot.title("Distance computations for neighbor {} and {}".format(FIRST_INDEX + 1, SECOND_INDEX + 1))

# Plot linear regression
reg = LinearRegression().fit(x.reshape(-1, 1), y)
plot.plot(x, reg.predict(x.reshape(-1, 1)), color='red')

if EXPORT_PNG:
    plot.savefig(VIS_DIR + "{}_vs_{}_neigh_found.png".format(FIRST_INDEX, SECOND_INDEX))
plot.show()
print("R^2: {}".format(reg.score(x.reshape(-1, 1), y)))